# Alberta Reports
All reports from alberta

In [56]:
import pandas as pd
import numpy as np
from os import listdir,environ,rename 
from sys import argv
from os.path import isfile,join,basename
from shutil import rmtree,move
import glob
from bs4 import BeautifulSoup
import os
from zipfile import ZipFile
import requests
from requests_html import HTMLSession,AsyncHTMLSession
import json
import urllib3

In [57]:
root_dir = os.path.abspath(os.curdir)
data_dir = join(root_dir,"data")
alberta_root_dir=join(data_dir,"01_raw","20210108","alberta")
reports_dir=join(alberta_root_dir,"reports")
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning) #Ignore the insercure warning smap.

In [58]:
MAKE_DIRS=False
GET_LINKS=False
DOWNLOAD = True
if MAKE_DIRS:
    os.makedirs(reports_dir)

In [59]:
def fxn():
    warnings.warn("https", InsecureRequestWarning)

def get_parsed_content(url):
    html_content = requests.get(url, verify=False).text
    soup = BeautifulSoup(html_content, "html.parser")
    return soup

def download_articles(article_links,http_root,destination_dir,max_length=100):
    for article_link in article_links:
        r = requests.get(article_link,stream=True, verify=False)
        chunk_size=10000000
        article_name = article_link.replace(http_root,"").replace("/","_")
        article_name = article_name[-1*max_length:]
        f = open(join(destination_dir,article_name),"wb+")
        for chunk in r.iter_content(chunk_size):
            #print("I\'m writing", chunk[0:10], "to", join(destination_dir,article_name))
            f.write(chunk)
        f.close()
                
def download_all_pdf(pdf_http_root, page_http_link, destination_dir):
    article_links= []
    for i in range(0,55):
        content=get_parsed_content(f'{page_http_link}{i}')
        for link in content.find_all('a', href=True):
            if ".pdf" in (href := link['href']):
                try:
                    download_articles([link['href']],pdf_http_root, destination_dir)
                except ValueError:
                    pass
        
    
def get_files_count(directory,print_caption=None):
    nb_files = len([name for name in os.listdir(directory) if os.path.isfile(directory +"/"+name)])
    if print_caption:
        print (f'{print_caption}: {nb_files}')
    return nb_files

In [60]:
download_all_pdf("https://static.ags.aer.ca/files/","https://ags.aer.ca/reports/reports?title=&publication_type=All&sort_by=created&sort_order=DESC&page=",alberta_root_dir)

In [61]:
get_files_count(alberta_root_dir,alberta_root_dir)

/home/ykinakin/Python/berkeley/geoscience_language_models/project_tools/data/01_raw/20201221/data/01_raw/20210108/alberta: 513


513